# AiiDA-JuTools Demo

Date: 2022-11-30.

Decription: Demo of [AiiDA-JuTools](https://github.com/JuDFTteam/aiida-jutools) for [JuDFTteam](https://github.com/JuDFTteam) AiiDA [hackathon 2022-12-01](https://iffmd.fz-juelich.de/ud8C294VQ5achi2GGj-D9A#). Intended to be run on [iffAiiDA](https://ifflogin.fz-juelich.de/), any AiiDA v1.x image there (for instance, `aiida-stable`, `aiida-develop`). Requires existence of AiiDA computer node `iffslurm` (IFF cluster).

## What is AiiDA-JuTools

![IAS-1 Software Stack](img/ias-1-software-stack/ias-1-software-stack.png)

**AiiDa-JuTools** is ...

* For AiiDA users in general: Make work with AiiDA simpler, faster, more effective.
* For AiiDA users at IAS-1: Avoid reinventing the wheel. Share common solutions.
* For developers at IAS-1: Share common solutions, for JuDFTteam AiiDA plugins.

## Setup notebook for the demo

### User-specific constants

Adjust if needed.

In [8]:
HOME_DIR = '~/Home/JupyterHub'
AIIDA_PROFILE_NAME = 'wasmer'

### Automatically set constants

In [30]:
# set by the notebook
JUTOOLS_REPO_DIR = '' # set dynamically below
JUTOOLS_DEMO_SUBDIR = '/notebooks/aiida-jutools-demo'

### Imports

In [5]:
from pathlib import Path
import os
import sys
import shutil

In [6]:
import aiida
from aiida.orm import QueryBuilder, Node

In [9]:
aiida.load_profile(AIIDA_PROFILE_NAME)

### Demo helper tools

In [3]:
def get_package_path(package) -> str:
    return os.path.dirname(package.__file__)

In [4]:
def list_package_modules(package) -> list:
    """Return list of modules inside an imported Python package."""
    from types import ModuleType
    
    if not isinstance(package, ModuleType):
        return []
    
    # reference: https://stackoverflow.com/a/1310912
    import os.path, pkgutil
    pkgpath = get_package_path(package)
    return [name for _, name, _ in pkgutil.iter_modules([pkgpath])]
    
    # # alternative, list includes __init__.py and such
    # return [path for path in os.listdir(os.path.dirname(jutools.__file__))]

## Getting AiiDA-JuTools

Google aiida-jutools, copy installation instruction from the README.

In [12]:
!pip install -e git+https://github.com/JuDFTteam/aiida-jutools@develop#egg=aiida-jutools

Obtaining aiida-jutools from git+https://github.com/JuDFTteam/aiida-jutools@develop#egg=aiida-jutools
  Cloning https://github.com/JuDFTteam/aiida-jutools (to revision develop) to /opt/aiida-kernel/src/aiida-jutools
  Running command git clone --filter=blob:none --quiet https://github.com/JuDFTteam/aiida-jutools /opt/aiida-kernel/src/aiida-jutools
  Running command git checkout -b develop --track origin/develop
  Switched to a new branch 'develop'
  Branch 'develop' set up to track remote branch 'develop' from 'origin'.
  Resolved https://github.com/JuDFTteam/aiida-jutools to commit 1cfc6734a98d55d1c57a22da322d08ee731d8b2d
  Preparing metadata (setup.py) ... done
  Running setup.py develop for aiida-jutools

[notice] A new release of pip available: 22.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


Before proceeding, restart notebook kernel for change to take effect.

Recommended way to import JuTools.

In [11]:
import aiida_jutools as jutools

Now copy the demo notebook to somehwere where you can open and run it.

In [34]:
JUTOOLS_REPO_DIR = Path(get_package_path(jutools)).parent

In [37]:
!cp -r {JUTOOLS_REPO_DIR}{JUTOOLS_DEMO_SUBDIR} {HOME_DIR}

Now you are all set up to run this locally on your own AiiDA database.

## What's in AiiDA-JuTools?

In [43]:
list_package_modules(jutools)

['_dev',
 'code',
 'computer',
 'group',
 'io',
 'logging',
 'meta',
 'node',
 'plugins',
 'process',
 'process_functions',
 'structure',
 'submit']

Type `jutools`, play around with tab completion on `jutools.`.

Time for a walkthrough of the JuTools modules!

## `.code` and `.computer`

In [43]:
jutools.computer.get_computers()

[<Computer: iffslurm (iffslurm), pk: 1>,
 <Computer: claix18 (localhost), pk: 2>,
 <Computer: localhost (iff734), pk: 3>,
 <Computer: iff734 (iff734), pk: 4>]

In [39]:
iffslurm = jutools.computer.get_computers('slurm')[0]

In [44]:
jutools.computer.shell_command(iffslurm, "whoami")

(0, 'wasmer\n', '')

In [40]:
jutools.computer.get_queues(iffslurm)

Idle nodes left on computer 'iffslurm': 59


[['th1', 24],
 ['th1-2020-32', 16],
 ['oscar', 9],
 ['th1-2020-gpu', 3],
 ['th1-2020-64', 3],
 ['viti', 2],
 ['th2-gpu', 2]]

In [41]:
jutools.code.get_code(computer_name_pattern='iff', code_name_pattern='kkr', queue_name='th1')

Idle nodes left on computer 'iffslurm': 59


<Code: Remote code 'kkrhost_3.5_intel' on iffslurm, pk: 148, uuid: 2e3a0d2b-6737-40aa-925a-cb1f03444ca4>

In [45]:
optman = jutools.computer.ComputerOptionsManager()

Info: ComputerOptionsManager: Call initialize() before use.


In [46]:
optman.initialize()

Initializing computer options configs: ['localhost', 'iffslurm', 'claix18', 'claix16'].
OptionsConfig 'localhost':
Loaded computer options groups: ['computer_options/localhost'].
Loaded 2, created 0 default computer options nodes. Use get_options() to load or create options nodes.
OptionsConfig 'iffslurm':
Loaded computer options groups: ['computer_options/iffslurm'].
Loaded 8, created 0 default computer options nodes. Use get_options() to load or create options nodes.
OptionsConfig 'claix18':
Loaded computer options groups: ['computer_options/claix18'].
Loaded 1, created 0 default computer options nodes. Use get_options() to load or create options nodes.
OptionsConfig 'claix16':
Loaded computer options groups: ['computer_options/claix16'].
Loaded 1, created 0 default computer options nodes. Use get_options() to load or create options nodes.


In [47]:
optman.iffslurm.computers

[<Computer: iffslurm (iffslurm), pk: 1>]

In [49]:
optman.iffslurm.options

[<Dict: uuid: 41ea72d7-2376-49d1-9110-b8a762e0ae19 (pk: 26323)>,
 <Dict: uuid: 10f58ca9-e1ea-4f48-8fd1-605be9df67c9 (pk: 26321)>,
 <Dict: uuid: 65f510dc-2755-418d-8e24-64b5be574a20 (pk: 26325)>,
 <Dict: uuid: 095cb3d9-041d-4d1e-bd6f-dd680cc78b00 (pk: 26319)>,
 <Dict: uuid: 3e4be4a6-60f5-41d2-8c94-ca8226887e87 (pk: 26320)>,
 <Dict: uuid: 62c8bf6b-ced7-4300-8d1d-4f46142f462a (pk: 26322)>,
 <Dict: uuid: 791c92e4-be98-4281-b95d-4dc247c5a9e5 (pk: 26318)>,
 <Dict: uuid: 452a56d2-30f5-435b-bf5d-4a1b73dc54df (pk: 26324)>]

In [50]:
options = optman.iffslurm.get_options(computer_name='iffslurm', queue_name='th1', account='wasmer')

Info: _OptionsConfig 'iffslurm', get_options(): 
Missing mandatory argument 'queue_name'. Try find matching computer and call get_queues().
Try to get computer from config's assigned computers.
Try to get computer from name pattern 'iffslurm'.
Idle nodes left on computer 'iffslurm': 59
Found queue_names '['th1', 'th1-2020-32', 'oscar', 'th1-2020-gpu', 'th1-2020-64', 'viti', 'th2-gpu']'.
Did not find specified computer options in config. Create options node and store.
Created and stored options  node(s) [<Dict: uuid: f6329f58-cd30-4e33-aed3-c0b0c24061f6 (pk: 26499)>].


In [52]:
options = options[0]

In [53]:
options.attributes

{'withmpi': True,
 'resources': {'num_machines': 1, 'tot_num_mpiprocs': 12},
 'queue_name': 'th1',
 'max_wallclock_seconds': 86400,
 'custom_scheduler_commands': '#SBATCH --account=wasmer'}

In [55]:
optman.get_help(mode='keywords')

{'options_fields': ['account',
  'additional_retrieve_list',
  'append_text',
  'custom_scheduler_commands',
  'environment_variables',
  'import_sys_environment',
  'input_filename',
  'max_memory_kb',
  'max_wallclock_seconds',
  'mpirun_extra_params',
  'output_filename',
  'parser_name',
  'prepend_text',
  'priority',
  'qos',
  'queue_name',
  'resources',
  'scheduler_stderr',
  'scheduler_stdout',
  'stash',
  'submit_script_filename',
  'withmpi'],
 'resources_fields': {'localhost': ['num_machines',
   'num_mpiprocs_per_machine',
   'num_cores_per_machine',
   'num_cores_per_mpiproc',
   'tot_num_mpiprocs'],
  'iffslurm': ['num_machines',
   'num_mpiprocs_per_machine',
   'num_cores_per_machine',
   'num_cores_per_mpiproc',
   'tot_num_mpiprocs'],
  'claix18': ['num_machines',
   'num_mpiprocs_per_machine',
   'num_cores_per_machine',
   'num_cores_per_mpiproc',
   'tot_num_mpiprocs'],
  'claix16': ['num_machines',
   'num_mpiprocs_per_machine',
   'num_cores_per_machine',
   

In [59]:
optman.get_help('descriptions', 'custom_scheduler_commands', 'submit_script_filename')

{'custom_scheduler_commands': {'name': 'custom_scheduler_commands',
  'required': 'False',
  'valid_type': "<class 'str'>",
  'help': 'Set a (possibly multiline) string with the commands that the user wants to manually set for the scheduler. The difference of this option with respect to the `prepend_text` is the position in the scheduler submission file where such text is inserted: with this option, the string is inserted before any non-scheduler command',
  'default': '',
  'non_db': 'True'},
 'submit_script_filename': {'name': 'submit_script_filename',
  'required': 'False',
  'valid_type': "<class 'str'>",
  'help': 'Filename to which the job submission script is written.',
  'default': '_aiidasubmit.sh',
  'non_db': 'True'}}

## `.group` and `.node`

## `.io`

## `.process` and `.submit`

## Miscellaneous: `.process_functions`, `.plugins`, `.meta`